In [1]:
from argparse import Namespace
from dataset import ReviewDataset
from classifier import BOWClassifier
import torch

/home/nik/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-fpb3uk5n because the default path (/home/nik/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
def use_cuda():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

args = Namespace (
    in_features=None,
    out_features=None,
    units_per_layer=[128, 64],
    dataset=None,
    optimizer="Adam",
    criterion="bce_logits",
    save_dir='../trained_models/bow.pth',
    batch_size=128,
    learning_rate=0.001,
    num_epochs=20,
    device=use_cuda()
)

In [3]:
args.dataset = ReviewDataset.load_dataset_and_make_vectorizer("../data/IMDB-dataset.csv")
args.in_features = len(args.dataset.get_vectorizer().review_vocab)
args.out_features = len(args.dataset.get_vectorizer().sentiment_vocab)

In [4]:
bow = BOWClassifier(args=args)
bow = bow.to(args.device)
bow.setup()
bow.fit()

100%|██████████| 20/20 [03:59<00:00, 11.96s/it]


In [5]:
loss, acc = bow.eval_net(mode='test')
print(loss)
print(acc)

0.26555460997116875
89.61338141025641
